In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier #Second is for data size >10k. CLASSIFICATION.
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor #as above. REGRESSION.
# import lightgbm

In [52]:
df = pd.read_csv('train_top_20k.csv')
df = df[df.columns.tolist()[1:]] #Drops the duplicated index values in the first column.
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


#### Some basic notes on the column definitions, units, etc:
- imbalance_size is defined as the "amount unmatched at the current reference price (in USD)"
- imbalance_buy_sell_flag gives the DIRECTION of the auction imbalance
- reference_price is the price at which paired shares are maximized, the imbalance is minimized and the distance from the bid-ask midpoint is minimized (in that order). **What does this mean exactly?**

In [53]:
weight_arr = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
stock_id_list = np.arange(200)
weights_df = pd.DataFrame({'stock_id':stock_id_list, 'weight':weight_arr})
df = df.merge(weights_df, how='left', on='stock_id')
df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,weight
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0,0.004
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1,0.001
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2,0.002
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3,0.006
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4,0.004


In [54]:
#Cell to explore the above loaded df if needed without disturbing what is below.

# df[~df['far_price'].isna()]

#### Next cell will be adding some BASIC features to the data as manual (domain-specific) augmentation. 
TBD: how to incorporate formal kernel methods if that is of interest.

In [55]:
#Add basics like categorical target (for classification tree), book skew + other simple data transforms.
df['book_skew'] = np.log(df['bid_size']/df['ask_size'])
df['target_cat'] = np.where(df['target'] < 0, -1, np.where(df['target'] > 0, 1, 0)) #should we make this binary?

#Add some moving-average style transformations of important variables.
ewma_windows = [5,20,30]
ewma_cols = ['imbalance_size', 'book_skew', 'bid_price', 'ask_price']

df.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id,weight,book_skew,target_cat
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0,0.004,1.965899,-1
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1,0.001,-1.852115,-1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2,0.002,0.692252,-1
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3,0.006,-5.328091,-1
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4,0.004,3.636964,-1


In [56]:
#Derive the train/test params for our toy model.
print(df.time_id.unique()) #gonna use up to 40 for training and the rest for testing

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52]


In [60]:
#Define the train-test split.
X_train = df[df['time_id'] <=35].drop(columns=['target', 'target_cat'])
X_train = X_train.fillna(1e-9) #For imputing NaN values. Does this value make a big difference to performance?
X_test = df[df['time_id'] >35].drop(columns=['target', 'target_cat'])
X_test = X_test.fillna(1e-9) #See above.
print("Xtrain length", len(X_train)); print("Xtest length", len(X_test))
y_train = df[df['time_id'] <=35]['target']
y_test = df[df['time_id'] >35]['target']
X_train.head()

Xtrain length 6876
Xtest length 3124


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,weight,book_skew
0,0,0,0,3180602.69,1,0.999812,13380276.64,1.000000e-09,1.000000e-09,0.999812,60651.50,1.000026,8493.03,1.0,0,0_0_0,0.004,1.965899
1,1,0,0,166603.91,-1,0.999896,1642214.25,1.000000e-09,1.000000e-09,0.999896,3233.04,1.000660,20605.09,1.0,0,0_0_1,0.001,-1.852115
2,2,0,0,302879.87,-1,0.999561,1819368.03,1.000000e-09,1.000000e-09,0.999403,37956.00,1.000298,18995.00,1.0,0,0_0_2,0.002,0.692252
3,3,0,0,11917682.27,-1,1.000171,18389745.62,1.000000e-09,1.000000e-09,0.999999,2324.90,1.000214,479032.40,1.0,0,0_0_3,0.006,-5.328091
4,4,0,0,447549.96,-1,0.999532,17860614.95,1.000000e-09,1.000000e-09,0.999394,16485.54,1.000016,434.10,1.0,0,0_0_4,0.004,3.636964


In [61]:
#Fit classifier.
clf = GradientBoostingRegressor(loss='absolute_error', learning_rate=0.05, n_estimators=1000, verbose=1, random_state=42)
clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           5.9358           27.29s
         2           5.9189           25.69s
         3           5.9033           23.47s
         4           5.8885           21.82s
         5           5.8748           20.80s
         6           5.8636           20.03s
         7           5.8523           19.49s
         8           5.8419           19.07s
         9           5.8324           18.79s
        10           5.8232           18.52s
        20           5.7502           17.19s
        30           5.6964           16.55s
        40           5.6555           16.22s
        50           5.6162           15.93s
        60           5.5750           15.70s
        70           5.5371           15.47s
        80           5.5131           15.28s
        90           5.5014           15.07s
       100           5.4755           14.87s
       200           5.2564           13.07s
       300           5.1551           11.45s
       40

GradientBoostingRegressor(learning_rate=0.05, loss='absolute_error',
                          n_estimators=1000, random_state=42, verbose=1)

In [64]:
#Run prediction.
y_pred = clf.predict(X_test)
prediction_mae = np.linalg.norm(y_pred-y_test,ord=1)/len(y_pred)
print(prediction_mae)s

4.798596976617157
